In [1]:
from ZW_utils import std_classes, dataloading
from ZW_dataset import PSI_Dataset
import numpy as np
from config import DATA_DIRECTORY
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
from split_functions import uniqueness_check


classes = std_classes
data_split_ratio = 0.85
batch_size = 8
max_epochs = 30
learning_rate = 1e-3
block_size = 22
n_embd = 32  # 32
n_head = 4  # 4
n_layer = 2  # 2
dropout = 0.1  # 0.1
vocab_size = len(classes)
criterion = nn.CrossEntropyLoss()

In [2]:
# layouts = np.load(DATA_DIRECTORY/"v22DF_m2_sorted_layouts.npy", allow_pickle=True)
# results = np.load(DATA_DIRECTORY/"v22DF_m2_sorted_results.npy", allow_pickle=True)
layouts = np.load("GPT_NA_psitest/M2_data_8_layouts.npy", allow_pickle=True)
results = np.load("GPT_NA_psitest/M2_data_8_results.npy", allow_pickle=True)
l2 = []
r2 = []
cutoff = 144
aug = True
for i, r in enumerate(results):
    if r > 0 and r < cutoff:
        l2.append(layouts[i])
        r2.append(r)
layouts = np.asanyarray(l2)
results = np.asanyarray(r2)
print("layouts", layouts.shape)
indices = np.argsort(results)
layouts = layouts[indices]
results = results[indices]
layouts = layouts[:]
results = results[:]
designs, equipments = uniqueness_check(layouts)
print(len(equipments),"before augmentation")
if aug == True:
    print("Augmenting")
    data = equipments
    results = results
    augmented_results = []
    augmented = []
    for design in data:
        base = np.array(design)
        base_result = results[data.index(design)]
        nognoe = base[1:-1]
        for j in range(1, len(nognoe)):
            new_rep = np.roll(nognoe, j, axis=0)
            augmented.append(
                np.concatenate((base[0:1], new_rep, base[-1:]), axis=0).tolist()
            )
            augmented_results.append(base_result)
    equipments = equipments + augmented
    results = np.concatenate((results, augmented_results), axis=0)
sorted_equipments = np.array(equipments.copy(),dtype=object)
index = np.argsort([len(e) for e in equipments])
sorted_equipments = sorted_equipments[index].tolist()
sorted_results = results[index]
print(len(sorted_equipments), len(results))
eq_array = np.zeros((len(sorted_equipments), 22))
for i, e in enumerate(sorted_equipments):
    for j, u in enumerate(e):
        eq_array[i, j] = u
re_array = np.array(sorted_results)
equipment_chunks = []
results_chunks = []
for equipment in sorted_equipments:
    for i in range(len(equipment)):
        candidate_chunk = equipment[: i + 1]
        if candidate_chunk not in equipment_chunks:
            equipment_chunks.append(candidate_chunk)
            # checking the same chunks in eq array
            chunk_indices = np.where(
                (eq_array[:, : i + 1] == candidate_chunk).all(axis=1)
            )[0]
            chunk_results = np.mean(re_array[chunk_indices])
            results_chunks.append(chunk_results)
print(25,equipment_chunks[25], results_chunks[25])
print(len(equipment_chunks), len(results_chunks))
print(25,equipment_chunks[25], results_chunks[25])

layouts (3121,)
3121 before augmentation
Augmenting
40584 40584
25 [0, 3, 5, 4, 1] 139.8402769721871
193389 193389
25 [0, 3, 5, 4, 1] 139.8402769721871


In [3]:
lengths = torch.tensor([x for x in map(len, equipment_chunks)])
max_length = max(lengths)
input_data = np.ones((len(equipment_chunks), max_length)) * 12
for i, e in enumerate(equipment_chunks):
    input_data[i, : len(e)] = e
input_data = torch.tensor(input_data)
target_data = torch.tensor(results_chunks).float().reshape(-1, 1)
print(input_data.shape, target_data.shape)

torch.Size([193389, 19]) torch.Size([193389, 1])


In [4]:
#classification of results into k categories
k = 10
class_limits = np.linspace(130, 145,k)
reg_classes = np.arange(k)
target_data = np.digitize(results_chunks, class_limits, right=True)
target_data = torch.tensor(target_data).long().reshape(-1)
reg_classes,target_data,class_limits
#number of instances per class
for i in range(k):
    print(i, len(target_data[target_data == i]),class_limits[i])
#examples from each class
indices = np.random.choice(len(target_data), 10)
print(target_data.dtype, input_data.dtype,lengths.dtype)
print(input_data.shape, target_data.shape, lengths.shape)


0 144 130.0
1 72 131.66666666666666
2 281 133.33333333333334
3 2379 135.0
4 15382 136.66666666666666
5 35530 138.33333333333334
6 48238 140.0
7 45406 141.66666666666666
8 34798 143.33333333333334
9 11159 145.0
torch.int64 torch.float64 torch.int64
torch.Size([193389, 19]) torch.Size([193389]) torch.Size([193389])


In [5]:
indices = torch.randperm(len(input_data))
input_data = input_data[indices]
target_data = target_data[indices]
lengths = lengths[indices]
train_data = input_data[: int(0.85 * len(input_data))]
train_target = target_data[: int(0.85 * len(input_data))]
train_lengths = lengths[: int(0.85 * len(input_data))]
val_data = input_data[int(0.85 * len(input_data)) :]
val_target = target_data[int(0.85 * len(input_data)) :]
val_lengths = lengths[int(0.85 * len(input_data)) :]
print(train_data[25], train_target[25], train_lengths[25])

tensor([ 0.,  9.,  2.,  1.,  3.,  2.,  7.,  3.,  5.,  4.,  7.,  1.,  5.,  2.,
         3., 12., 12., 12., 12.], dtype=torch.float64) tensor(6) tensor(15)


In [6]:
class MLP(nn.Module):
    def __init__(self, hidden_size):
        super(MLP, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(21, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, 1),
        )

    def forward(self, x):
        mask = (x != 12).float()
        x = x * mask
        x = self.model(x)
        return x


# LSTM model with masked input where the token is 12 (padding)
class LSTM(nn.Module):
    def __init__(self, hidden_size):
        super(LSTM, self).__init__()
        self.lstm = nn.LSTM(21, hidden_size, num_layers=2, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        mask = (x != 12).float()
        x = x * mask
        x, _ = self.lstm(x)
        x = self.fc(x)
        return x


class LSTM_packed(nn.Module):
    def __init__(self, embd_size,hidden_size,k):
        super(LSTM_packed, self).__init__()
        self.embedding = nn.Embedding(13, embd_size)
        self.lstm = nn.LSTM(embd_size, hidden_size, num_layers=2, batch_first=True, dropout=0.1)
        self.fc = nn.Linear(hidden_size, k)

    def forward(self, x, lengths):
        x = self.embedding(x.long())
        x = nn.utils.rnn.pack_padded_sequence(
            x, lengths, batch_first=True, enforce_sorted=False
        )
        output, (hidden, _) = self.lstm(x)
        x = self.fc(hidden[-1])
        return x

In [7]:
model = LSTM_packed(64,256,k)
trial_train_data = train_data[:8]
trial_train_lengths = train_lengths[:8]
trial_train_target = train_target[:8]
output = model(trial_train_data, trial_train_lengths)
print(output)
loss = criterion(output, trial_train_target)
picks = torch.argmax(output, dim=1)
print(picks, trial_train_target)
print(len(train_data),len(val_data))

tensor([[-0.0281, -0.0755,  0.0280, -0.0215, -0.0361, -0.0122,  0.0378, -0.0281,
          0.0233,  0.0138],
        [-0.0093, -0.0732,  0.0255, -0.0292, -0.0397, -0.0112,  0.0424, -0.0322,
          0.0253,  0.0177],
        [-0.0188, -0.0731,  0.0278, -0.0065, -0.0406, -0.0111,  0.0433, -0.0323,
          0.0185,  0.0185],
        [-0.0271, -0.0697,  0.0243, -0.0266, -0.0379, -0.0207,  0.0354, -0.0426,
          0.0161,  0.0147],
        [-0.0212, -0.0694,  0.0329, -0.0128, -0.0306, -0.0106,  0.0450, -0.0471,
          0.0183,  0.0118],
        [-0.0056, -0.0616,  0.0342, -0.0200, -0.0304, -0.0066,  0.0392, -0.0464,
          0.0103,  0.0220],
        [-0.0338, -0.0708,  0.0290, -0.0159, -0.0389, -0.0207,  0.0383, -0.0403,
          0.0192,  0.0127],
        [-0.0030, -0.0616,  0.0415, -0.0170, -0.0323, -0.0079,  0.0351, -0.0499,
          0.0233,  0.0104]], grad_fn=<AddmmBackward0>)
tensor([6, 6, 6, 6, 6, 6, 6, 2]) tensor([8, 7, 7, 9, 6, 9, 9, 6])
164380 29009


In [8]:
for embd_size in [64]:
    for hidden_size in [256]:
        model_name = f"psi_aug_classification_{embd_size}_{hidden_size}_{batch_size}_{cutoff}_8.pt"
        batch_size = 4
        patience = 10
        model = LSTM_packed(embd_size,hidden_size,k)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        best_loss = 1e9
        for epoch in range(max_epochs+1):
            model.train()
            epoch_loss = 0
            for i in range(0, len(train_data), batch_size):
                input_batch = train_data[i : i + batch_size]
                target_batch = train_target[i : i + batch_size]
                lengths_batch = train_lengths[i : i + batch_size]
                optimizer.zero_grad()
                output = model(input_batch, lengths_batch)
                loss = criterion(output, target_batch)
                loss.backward()
                optimizer.step()
                epoch_loss += loss.item()
            epoch_loss /= len(train_data) / batch_size
            
            indices = torch.randperm(len(train_data))
            train_data = train_data[indices]
            train_target = train_target[indices]
            train_lengths = train_lengths[indices]

            model.eval()
            val_loss = 0
            with torch.no_grad():
                for i in range(0, len(val_data), batch_size):
                    input_batch = val_data[i : i + batch_size]
                    target_batch = val_target[i : i + batch_size]
                    lengths_batch = val_lengths[i : i + batch_size]
                    output = model(input_batch, lengths_batch)
                    loss = criterion(output, target_batch)
                    val_loss += loss.item()
                val_loss /= len(val_data) / batch_size
            if val_loss < best_loss:
                best_model_epoch = epoch
                best_loss = val_loss
                best_model = model.state_dict()
                patience = 10
            else:
                patience -= 1
            if patience == 0:
                break
            print(f"Epoch {epoch} Training Loss: {epoch_loss:.2f} Validation Loss: {val_loss:.2f}")
        torch.save(best_model, model_name)

Epoch 0 Training Loss: 1.26 Validation Loss: 0.95
Epoch 1 Training Loss: 0.80 Validation Loss: 0.71
Epoch 2 Training Loss: 0.64 Validation Loss: 0.60
Epoch 3 Training Loss: 0.57 Validation Loss: 0.58
Epoch 4 Training Loss: 0.53 Validation Loss: 0.54
Epoch 5 Training Loss: 0.51 Validation Loss: 0.54
Epoch 6 Training Loss: 0.51 Validation Loss: 0.54
Epoch 7 Training Loss: 0.51 Validation Loss: 0.53
Epoch 8 Training Loss: 0.51 Validation Loss: 0.54
Epoch 9 Training Loss: 0.52 Validation Loss: 0.55
Epoch 10 Training Loss: 0.52 Validation Loss: 0.55
Epoch 11 Training Loss: 0.53 Validation Loss: 0.55
Epoch 12 Training Loss: 0.54 Validation Loss: 0.55
Epoch 13 Training Loss: 0.54 Validation Loss: 0.57
Epoch 14 Training Loss: 0.56 Validation Loss: 0.58
Epoch 15 Training Loss: 0.57 Validation Loss: 0.59
Epoch 16 Training Loss: 0.57 Validation Loss: 0.60


In [9]:
model.state_dict(torch.load(model_name))
model.eval()
#random prediction / choose 5 random examples from the validation set
indices = torch.randperm(len(val_data))
val_data = val_data[indices]
val_target = val_target[indices]
val_lengths = val_lengths[indices]
val_data = val_data[:5]
val_target = val_target[:5]
val_lengths = val_lengths[:5]
total_predictions = 0
correct_predictions = 0
output = model(val_data, val_lengths)
picks = torch.argmax(output, dim=1)
print("Predictions", picks)
print("Targets", val_target)

Predictions tensor([9, 8, 6, 7, 8])
Targets tensor([8, 8, 5, 7, 8])


c:\Users\m85830ak\Python\Python-Code\.venv\Lib\site-packages\torch\nn\modules\module.py:1877: UserWarning: Positional args are being deprecated, use kwargs instead. Refer to https://pytorch.org/docs/master/generated/torch.nn.Module.html#torch.nn.Module.state_dict for details.
  warnings.warn(


In [10]:
# # best model prediction and mean error
# embd_size = 128
# hidden_size = 1024
# batch_size = 100
# cutoff = 143.957
# model = LSTM_packed(embd_size,hidden_size)
# model.load_state_dict(torch.load(f"psi_norm_{embd_size}_{hidden_size}_100_1.pt"))
# model.eval()
# print("Best Model Prediction",embd_size,hidden_size)
# # print("batch_size",batch_size,"found epoch",best_model_epoch)
# mean_error = 0
# with torch.no_grad():
#     for i in range(0, len(input_data)):
#         input_batch = input_data
#         target_batch = target_data
#         lengths_batch = lengths
#         output = model(input_batch, lengths_batch)
#         if target_batch.item() == 0:
#             continue
#         mean_error += ((torch.abs(output - target_batch))/torch.abs(target_batch)).item()
#     mean_error /= len(input_data)
#     print(f"Mean Error: {mean_error*100:.2f}%")


#     mean_val = np.mean(val_target.numpy())
#     predicted = np.zeros(len(val_data))
#     ssres = 0 
#     for i in range(len(val_data)):
#         # random_index = np.random.randint(0, len(val_data))
#         random_index = i
#         random_input = val_data[random_index]
#         random_target = val_target[random_index]
#         random_length = val_lengths[random_index]
#         random_output = model(random_input.unsqueeze(0), random_length.unsqueeze(0))
#         predicted[i] = random_output.item()
#         ssres += (random_target - random_output)**2
#         # print(f"Target: {random_target.item():.2f} Prediction: {random_output.item():.2f} Error: {abs((random_target.item() - random_output.item())/random_target.item())*100:.2f}")
#     sstot = np.sum((val_target.numpy() - mean_val)**2)
#     r2 = 1 - ssres/sstot
#     print("R2",r2)